In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn; seaborn.set()

In [2]:
Beads_range=[x for x in range(10,50,4) if x not in (38,46)]
Frac_range=[x/100 for x in range(0,100,25) if x!=0]
Dens_range=[x/100 for x in range(0,81,5) if x>0]

In [3]:
def Data(bead, frac, dens):
    data = pd.read_table('frac_'+str(frac)+'/m_'+str(bead)+'/d_'+str(dens)+'/final_snapshot.xyz', sep='\s+', header = 1)
    X = np.array([data.loc[data.type == 1, 'x'][part:part+bead].mean() for part in range(0,bead*1000,bead)])
    Y = np.array([data.loc[data.type == 1, 'y'][part:part+bead].mean() for part in range(0,bead*1000,bead)])
    Z = np.array([data.loc[data.type == 1, 'z'][part:part+bead].mean() for part in range(0,bead*1000,bead)])
    box_L = ((bead*1000)/dens)**(1./3.)
    return X, Y, Z, box_L

In [4]:
def pairCorrelationFunction(x, y, z, box_L, r_Max, cuts, dens):

    # Find particles which are close enough to the cube center that a sphere of radius
    # rMax will not cross any face of the cube
    bools1 = x > r_Max
    bools2 = x < (box_L - r_Max)
    bools3 = y > r_Max
    bools4 = y < (box_L - r_Max)
    bools5 = z > r_Max
    bools6 = z < (box_L - r_Max)

    reference_indices, = np.where(bools1 * bools2 * bools3 * bools4 * bools5 * bools6)
    num_interior_particles = len(reference_indices)

    if num_interior_particles < 1:
        raise  RuntimeError ("No particles found for which a sphere of radius rMax\
                will lie entirely within a cube of side length S.  Decrease rMax\
                or increase the size of the cube.")
    
    
    dr = r_Max / cuts
    edges = np.arange(0., r_Max + 1.1 * dr, dr)
    num_increments = len(edges) - 1
    g = np.zeros([num_interior_particles, num_increments])
    radii = np.zeros(num_increments)

    # Compute pairwise correlation for each interior particle
    for p in range(num_interior_particles):
        index = reference_indices[p]
        d = np.sqrt((x[index] - x)**2 + (y[index] - y)**2 + (z[index] - z)**2)
        d[index] = 2 * r_Max

        (result, bins) = np.histogram(d, bins=edges, normed=False)
        g[p,:] = result / dens

    # Average g(r) for all interior particles and compute radii
    g_average = np.zeros(num_increments)
    for i in range(num_increments):
        radii[i] = (edges[i] + edges[i+1]) / 2.
        rOuter = edges[i + 1]
        rInner = edges[i]
        g_average[i] = np.mean(g[:, i]) / (4.0 / 3.0 * np.pi * (rOuter**3 - rInner**3))

    return (g_average, radii, reference_indices)
    # Number of particles in shell/total number of particles/volume of shell/number density
    # shell volume = 4/3*pi(r_outer**3-r_inner**3)

In [6]:
def RDFer(Beads,Fracs,Denss):
    
    for Bead in Beads:
            
            for Frac in Fracs:
                
                frac = int((1-Frac)*10%10)
                
                for Dens in Denss:
                    
                    x_cm, y_cm, z_cm, L = Data(Bead,Frac,Dens)
                    
                    g_r, r, reference_indices = pairCorrelationFunction(x_cm, y_cm, z_cm, L, 3*L/7, 420, Dens)
                    print(r[0],r[-1],len(r))
                    
#                     plt.figure()
#                     plt.plot(r, g_r)
#                     plt.title('RDF of phobic blocks\'s center of mass; f ='+str(frac))
#                     plt.xlabel('r')
#                     plt.ylabel('g(r)')
#                     plt.xlim( (0, L/2.) )
#                     plt.ylim( (0, 1.05 * g_r.max()) )
#                     plt.show()
                    
                    print(Bead, Frac, Dens,'thank you, next')

In [7]:
RDFer(Beads_range,Frac_range,Dens_range)

C:\Users\mksal\anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater
  """
C:\Users\mksal\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in less
  
C:\Users\mksal\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in greater
  import sys
C:\Users\mksal\anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in less
  
C:\Users\mksal\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':
C:\Users\mksal\anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in less
  # Remove the CWD from sys.path while we load stuff.
C:\Users\mksal\anaconda3\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: Passing normed=False is deprecated, and has no effect. Consider passing the density a

0.029836915696049647 25.092846100377756 421
10 0.25 0.05 thank you, next
0.023681575681697847 19.91620514830789 421
10 0.25 0.1 thank you, next
0.02068776188970544 17.398407749242274 421
10 0.25 0.15 thank you, next
0.018796079074695847 15.807502501819208 421
10 0.25 0.2 thank you, next
0.01744873414976221 14.674385419950017 421
10 0.25 0.25 thank you, next
0.016419887493293022 13.809125381859431 421
10 0.25 0.3 thank you, next
0.015597485139432642 13.117485002262852 421
10 0.25 0.35 thank you, next
0.014918457848024825 12.546423050188878 421
10 0.25 0.4 thank you, next
0.014344092947308658 12.06338216868658 421
10 0.25 0.45 thank you, next
0.013849069472422992 11.647067426307736 421
10 0.25 0.5 thank you, next
0.013415999460863748 11.282855546586411 421
10 0.25 0.55 thank you, next
0.013032473340026413 10.960310078962213 421
10 0.25 0.6 thank you, next
0.012689352924475317 10.671745809483742 421
10 0.25 0.65 thank you, next
0.012379732159196867 10.411354745884566 421
10 0.25 0.7 thank

0.022864309931414105 19.228884652319262 421
18 0.5 0.2 thank you, next
0.021225345133182694 17.850515257006645 421
18 0.5 0.25 thank you, next
0.019973814495759502 16.79797799093374 421
18 0.5 0.3 thank you, next
0.018973411048196737 15.956638691533456 421
18 0.5 0.35 thank you, next
0.018147414818826847 15.261975862633378 421
18 0.5 0.4 thank you, next
0.01744873414976221 14.674385419950017 421
18 0.5 0.45 thank you, next
0.016846567596401157 14.167963348573373 421
18 0.5 0.5 thank you, next
0.016319763738695432 13.724921304242859 421
18 0.5 0.55 thank you, next
0.015853227071193153 13.33256396687344 421
18 0.5 0.6 thank you, next
0.015435841535955787 12.981542731738816 421
18 0.5 0.65 thank you, next
0.015059206328666279 12.66479252240834 421
18 0.5 0.7 thank you, next
0.014716832350075594 12.376856006413576 421
18 0.5 0.75 thank you, next
0.014403612686954516 12.113438269728746 421
18 0.5 0.8 thank you, next
0.036294829637653694 30.523951725266755 421
18 0.75 0.05 thank you, next
0.

0.021447616219885397 18.037445240923617 421
26 0.75 0.35 thank you, next
0.020513906931576825 17.25219572945611 421
26 0.75 0.4 thank you, next
0.019724115638261888 16.58798125177825 421
26 0.75 0.45 thank you, next
0.01904342426947575 16.015519810629105 421
26 0.75 0.5 thank you, next
0.0184479231793172 15.514703393805767 421
26 0.75 0.55 thank you, next
0.017920548350844064 15.071181163059858 421
26 0.75 0.6 thank you, next
0.01744873414976221 14.674385419950017 421
26 0.75 0.65 thank you, next
0.017022984274828148 14.316329775130473 421
26 0.75 0.7 thank you, next
0.016635963423499142 13.990845239162777 421
26 0.75 0.75 thank you, next
0.01628189872158139 13.69307682484995 421
26 0.75 0.8 thank you, next
0.04303227884192597 36.19014650605973 421
30 0.25 0.05 thank you, next
0.03415474235113109 28.724138317301247 421
30 0.25 0.1 thank you, next
0.029836915696049647 25.092846100377756 421
30 0.25 0.15 thank you, next
0.027108636968944153 22.798363690882034 421
30 0.25 0.2 thank you, n

0.022344485407591264 18.791712227784252 421
42 0.25 0.5 thank you, next
0.02164575784520741 18.20408234781943 421
42 0.25 0.55 thank you, next
0.021026965815349772 17.683678250709157 421
42 0.25 0.6 thank you, next
0.02047336550786379 17.21810039211345 421
42 0.25 0.65 thank you, next
0.019973814495759502 16.79797799093374 421
42 0.25 0.7 thank you, next
0.01951970594665703 16.41607270113856 421
42 0.25 0.75 thank you, next
0.019104266293924758 16.066687953190723 421
42 0.25 0.8 thank you, next
0.04813973449302584 40.485516708634734 421
42 0.5 0.05 thank you, next
0.038208532587849516 32.133375906381445 421
42 0.5 0.1 thank you, next
0.033378227654985604 28.07108945784289 421
42 0.5 0.15 thank you, next
0.03032613241205678 25.504277358539753 421
42 0.5 0.2 thank you, next
0.028152287514093065 23.676073799352267 421
42 0.5 0.25 thank you, next
0.026492316846179097 22.280038467636622 421
42 0.5 0.3 thank you, next
0.025165429329902754 21.164126066448215 421
42 0.5 0.35 thank you, next
0.

KeyboardInterrupt: 